# Reconstruction of the CHX dataset

We will poerform the same reconstruction as we did for the untreated dataset.
That means starting with particle extraction and making star files.
The CHX angles are more varied (and not as on axis) as the untreated cells. We will therefore just use the first angular search of psi angles to sort the particles to rotated and unrotated.

In [2]:
import pandas as pd
import mrcfile
import numpy as np
import os
from pathlib import Path
import glob
import starfile
import sys
import random

# Set up the command line arguments
sys.argv = [
    'export_relion_constrained.py',
    'CHX_data/results_all_steps_4/step_1/',
    'CHX_data/results_refine_tm_60S/',
    'CHX_data/extracted_60S_constrained_step1/',
    '--box-size', '512',
    '--csv-suffix-60s', '_refined_results.csv',
    '--csv-suffix-40s', '_constrained_results_above_threshold.csv'
]

def wrap_angle(angle):
    """Wrap angle from 0-360 to -180 to 180 degrees"""
    return (angle + 180) % 360 - 180

def create_star_file(df, output_dir, output_star_path, box_size) -> None:
    """
    Create a particles.star file from the particle data.
    
    Args:
        df (pandas.DataFrame): DataFrame containing particle data
        output_dir (str): Directory containing the particle stacks
        output_star_path (str): Path to save the star file
        box_size (int): Size of the box used for particle extraction
    """
    # Create a new DataFrame for particles
    particle_df = pd.DataFrame()
    
    # Group by micrograph to get particle indices
    for micrograph_path, particles in df.groupby('micrograph_path'):
        mrcs_name = f"{Path(micrograph_path).stem}.mrcs"
        mrcs_rel_path = os.path.join(output_dir, mrcs_name)
        
        # Add each particle to the particle DataFrame
        for _, particle in particles.iterrows():
            particle_data = {
                'rlnCoordinateX': particle['pos_x'],
                'rlnCoordinateY': particle['pos_y'],
                'rlnImageName': f"{particle['stack_order']:06d}@{mrcs_rel_path}",
                'rlnMicrographName': micrograph_path,  # Use full path
                'rlnOpticsGroup': 1,
                'rlnDefocusU': particle['defocus_u'] + particle['refined_relative_defocus'],
                'rlnDefocusV': particle['defocus_v'] + particle['refined_relative_defocus'],
                'rlnDefocusAngle': particle['astigmatism_angle'],
                'rlnAngleRot': wrap_angle(particle['refined_phi']),
                'rlnAngleTilt': particle['refined_theta'],
                'rlnAnglePsi': wrap_angle(particle['refined_psi']),
                'rlnRandomSubset': random.randint(1, 2)
            }
            particle_df = pd.concat([particle_df, pd.DataFrame([particle_data])], ignore_index=True)
    
    # Create optics DataFrame
    optics_data = {
        'rlnOpticsGroup': 1,
        'rlnOpticsGroupName': 'OpticsGroup1',
        'rlnMicrographOriginalPixelSize': df['refined_pixel_size'].iloc[0],
        'rlnSphericalAberration': df['spherical_aberration'].iloc[0],
        'rlnVoltage': df['voltage'].iloc[0],
        'rlnAmplitudeContrast': df['amplitude_contrast_ratio'].iloc[0],
        'rlnImagePixelSize': df['refined_pixel_size'].iloc[0],
        'rlnImageSize': box_size
    }
    optics_df = pd.DataFrame([optics_data])
    
    # Write the star file
    starfile.write(
        {'optics': optics_df, 'particles': particle_df},
        output_star_path
    )
    print(f"Created star file: {output_star_path}")

def create_particle_stacks(input_40s_dir, input_60s_dir, output_dir, box_size, sort_column='original_offset_psi', 
                         csv_suffix_40s='_results.csv', csv_suffix_60s='_results.csv') -> None:
    """
    Create particle stacks from CSV files containing particle positions.
    
    Args:
        input_40s_dir (str): Directory containing 40S input CSV files
        input_60s_dir (str): Directory containing 60S input CSV files
        output_dir (str): Directory to save the output stacks
        box_size (int): Size of the square box to crop around each particle
        sort_column (str): Column name to use for sorting particles (default: 'original_offset_psi')
        csv_suffix_40s (str): Suffix pattern to match 40S CSV files (default: '_results.csv')
        csv_suffix_60s (str): Suffix pattern to match 60S CSV files (default: '_results.csv')
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Find all CSV files in both directories
    csv_pattern_40s = os.path.join(input_40s_dir, f'*{csv_suffix_40s}')
    csv_pattern_60s = os.path.join(input_60s_dir, f'*{csv_suffix_60s}')
    csv_files_40s = glob.glob(csv_pattern_40s)
    csv_files_60s = glob.glob(csv_pattern_60s)
    
    if not csv_files_40s or not csv_files_60s:
        print("No CSV files found in one or both input directories")
        return
    
    print(f"Found {len(csv_files_40s)} 40S CSV files and {len(csv_files_60s)} 60S CSV files to process")
    
    # Process 40S particles first to get their states
    forty_s_particles = {}
    for csv_path in csv_files_40s:
        df = pd.read_csv(csv_path)
        for _, particle in df.iterrows():
            micrograph_name = Path(particle['micrograph_path']).stem
            key = (micrograph_name, particle['particle_index'])
            forty_s_particles[key] = particle[sort_column]
    
    # Initialize DataFrames to store particles for each category
    category_dfs = {
        '60S_only': [],
        '40S_nonrotated': [],
        '40S_rotated': [],
        'rest': []  # Add category for particles that don't fit other categories
    }
    
    # Process 60S particles
    for csv_path in csv_files_60s:
        print(f"\nProcessing CSV file: {csv_path}")
        
        # Read the CSV file
        df = pd.read_csv(csv_path)
        
        # Group particles by micrograph
        micrograph_groups = df.groupby('micrograph_path')
        
        for micrograph_path, particles in micrograph_groups:
            print(f"Processing micrograph: {micrograph_path}")
            
            # Get pixel size from first particle in the group
            pixel_size = particles.iloc[0]['refined_pixel_size']
            
            # Load the micrograph
            with mrcfile.open(micrograph_path) as mrc:
                micrograph = mrc.data
            
            # Initialize lists to store particles and their metadata
            particle_data = []
            particle_categories = []  # Store which category each particle belongs to
            
            # Process each particle
            for _, particle in particles.iterrows():
                # Get particle top-left coordinates
                x_start = int(particle['pos_x'])
                y_start = int(particle['pos_y'])
                
                # Calculate box boundaries
                x_end = min(micrograph.shape[-1], x_start + box_size)
                y_end = min(micrograph.shape[-2], y_start + box_size)
                
                # Skip if the box is empty or invalid
                if x_start >= x_end or y_start >= y_end:
                    print(f"Warning: Invalid box coordinates for particle at ({x_start}, {y_start})")
                    continue
                
                # Extract particle
                particle_img = micrograph[..., y_start:y_end, x_start:x_end]
                
                # Create padded image
                padded_img = np.zeros((box_size, box_size), dtype=micrograph.dtype)
                
                # Calculate the region to copy
                copy_height = min(particle_img.shape[-2], box_size)
                copy_width = min(particle_img.shape[-1], box_size)
                
                # Copy the valid region
                padded_img[:copy_height, :copy_width] = particle_img[..., :copy_height, :copy_width]
                
                # Check 40S state and determine category
                micrograph_name = Path(micrograph_path).stem
                key = (micrograph_name, particle['particle_index'])
                
                category = '60S_only'  # Default category
                if key in forty_s_particles:
                    psi_value = forty_s_particles[key]
                    if -2.0 <= psi_value <= 2.0:
                        category = '40S_nonrotated'
                    elif -12.0 <= psi_value <= -4.5:
                        category = '40S_rotated'
                    else:
                        category = 'rest'  # 40S particle exists but psi is outside our ranges
                
                # Store particle data and its category
                particle_data.append({
                    'image': padded_img,
                    'metadata': particle
                })
                particle_categories.append(category)
            
            if not particle_data:
                print(f"Warning: No valid particles found in {micrograph_path}")
                continue
            
            # Create single stack for all particles
            particle_stack = np.stack([p['image'] for p in particle_data])
            
            # Create output filename (just one stack per micrograph)
            micrograph_name = Path(micrograph_path).stem
            output_path = os.path.join(output_dir, f"{micrograph_name}.mrcs")
            
            # Save the stack
            with mrcfile.new(output_path, overwrite=True) as mrc:
                mrc.set_data(particle_stack)
                mrc.voxel_size = pixel_size
            
            # Add particles to their respective category DataFrames with their actual stack positions
            for idx, (particle, category) in enumerate(zip(particle_data, particle_categories)):
                particle_df = pd.DataFrame([particle['metadata']])
                particle_df['stack_order'] = idx + 1  # Position in the actual stack
                category_dfs[category].append(particle_df)
    
    # Create star files for each category at the end
    for category, dfs in category_dfs.items():
        if dfs:
            all_particles_df = pd.concat(dfs, ignore_index=True)
            star_file_path = os.path.join(output_dir, f'particles_{category}.star')
            create_star_file(all_particles_df, output_dir, star_file_path, box_size)

    # Create a star file for all particles combined
    all_particles_combined = []
    for dfs in category_dfs.values():
        if dfs:
            all_particles_combined.extend(dfs)

    if all_particles_combined:
        all_particles_df = pd.concat(all_particles_combined, ignore_index=True)
        star_file_path = os.path.join(output_dir, 'particles_all.star')
        create_star_file(all_particles_df, output_dir, star_file_path, box_size)
        print(f"Created combined star file with all particles: {star_file_path}")

if __name__ == "__main__":
    import argparse
    
    parser = argparse.ArgumentParser(description="Process and sort 40S and 60S particles")
    parser.add_argument("input_40s_dir", help="Directory containing 40S input CSV files")
    parser.add_argument("input_60s_dir", help="Directory containing 60S input CSV files")
    parser.add_argument("output_dir", help="Directory to save the output stacks and star files")
    parser.add_argument("--box-size", type=int, default=256, help="Size of the square box to crop around each particle")
    parser.add_argument("--sort-column", default='original_offset_psi',
                      help="Column name to use for sorting particles (default: original_offset_psi)")
    parser.add_argument("--csv-suffix-40s", default='_results.csv',
                      help="Suffix pattern to match 40S CSV files (default: _results.csv)")
    parser.add_argument("--csv-suffix-60s", default='_results.csv',
                      help="Suffix pattern to match 60S CSV files (default: _results.csv)")
    
    args = parser.parse_args()
    
    create_particle_stacks(args.input_40s_dir, args.input_60s_dir, args.output_dir, 
                         args.box_size, args.sort_column, 
                         args.csv_suffix_40s, args.csv_suffix_60s) 

Found 35 40S CSV files and 44 60S CSV files to process

Processing CSV file: CHX_data/results_refine_tm_60S/146_Sep13_10.20.11_249_1_refined_results.csv
Processing micrograph: /home/data/jdickerson/Leopard-EM_paper_data/CHX_data/all_mgraphs/146_Sep13_10.20.11_249_1.mrc

Processing CSV file: CHX_data/results_refine_tm_60S/82_Sep12_16.22.05_175_1_refined_results.csv
Processing micrograph: /home/data/jdickerson/Leopard-EM_paper_data/CHX_data/mgraphs_25/82_Sep12_16.22.05_175_1.mrc

Processing CSV file: CHX_data/results_refine_tm_60S/107_Sep12_18.57.58_187_1_refined_results.csv
Processing micrograph: /home/data/jdickerson/Leopard-EM_paper_data/CHX_data/mgraphs_25/107_Sep12_18.57.58_187_1.mrc

Processing CSV file: CHX_data/results_refine_tm_60S/131_Sep13_10.52.39_221_1_refined_results.csv
Processing micrograph: /home/data/jdickerson/Leopard-EM_paper_data/CHX_data/mgraphs_25/131_Sep13_10.52.39_221_1.mrc

Processing CSV file: CHX_data/results_refine_tm_60S/147_Sep13_11.27.34_253_1_refined_resu

We will again reconstruct these with RELION

In [3]:
!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_60S_only.star --o CHX_data/reconstructions/reconstruct_60S-only_half1.mrc --ctf --subset 1
!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_60S_only.star --o CHX_data/reconstructions/reconstruct_60S-only_half2.mrc --ctf --subset 2
# Next, we'll run the following command to generate a 3D volume from the 60S particles with rotated 40S
!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_40S_rotated.star --o CHX_data/reconstructions/reconstruct_60S-rotated_half1.mrc --ctf --subset 1
!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_40S_rotated.star --o CHX_data/reconstructions/reconstruct_60S-rotated_half2.mrc --ctf --subset 2

# Next, we'll run the following command to generate a 3D volume from the 60S particles with non-rotated 40S
!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_40S_nonrotated.star --o CHX_data/reconstructions/reconstruct_60S-nonrotated_half1.mrc --ctf --subset 1
!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_40S_nonrotated.star --o CHX_data/reconstructions/reconstruct_60S-nonrotated_half2.mrc --ctf --subset 2

!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_all.star --o CHX_data/reconstructions/reconstruct_60S-all_half1.mrc --ctf --subset 1
!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_all.star --o CHX_data/reconstructions/reconstruct_60S-all_half2.mrc --ctf --subset 2



 + Taking data dimensions from the first optics group: 2
 + Taking angpix from the first optics group: 1.059
 + Back-projecting all images ...
1.45/1.45 min ............................................................~~(,_,">
 + Starting the reconstruction ...
 + Done! Written output map in: CHX_data/reconstructions/reconstruct_60S-only_half1.mrc
 + Taking data dimensions from the first optics group: 2
 + Taking angpix from the first optics group: 1.059
 + Back-projecting all images ...
1.48/1.48 min ............................................................~~(,_,">
 + Starting the reconstruction ...
 + Done! Written output map in: CHX_data/reconstructions/reconstruct_60S-only_half2.mrc
 + Taking data dimensions from the first optics group: 2
 + Taking angpix from the first optics group: 1.059
 + Back-projecting all images ...
  20/  20 sec ............................................................~~(,_,">
 + Starting the reconstruction ...
 + Done! Written output map in: CHX_data/

We will also reconstruct all as well as subsets

In [4]:
!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_60S_only.star --o CHX_data/reconstructions/reconstruct_60S-only_all.mrc --ctf
# Next, we'll run the following command to generate a 3D volume from the 60S particles with rotated 40S
!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_40S_rotated.star --o CHX_data/reconstructions/reconstruct_60S-rotated_all.mrc --ctf

# Next, we'll run the following command to generate a 3D volume from the 60S particles with non-rotated 40S
!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_40S_nonrotated.star --o CHX_data/reconstructions/reconstruct_60S-nonrotated_all.mrc --ctf

!relion_reconstruct --i CHX_data/extracted_60S_constrained_step1/particles_all.star --o CHX_data/reconstructions/reconstruct_60S-all_all.mrc --ctf



 + Taking data dimensions from the first optics group: 2
 + Taking angpix from the first optics group: 1.059
 + Back-projecting all images ...
3.13/3.13 min ............................................................~~(,_,">
 + Starting the reconstruction ...
 + Done! Written output map in: CHX_data/reconstructions/reconstruct_60S-only_all.mrc
 + Taking data dimensions from the first optics group: 2
 + Taking angpix from the first optics group: 1.059
 + Back-projecting all images ...
  39/  39 sec ............................................................~~(,_,">
 + Starting the reconstruction ...
 + Done! Written output map in: CHX_data/reconstructions/reconstruct_60S-rotated_all.mrc
 + Taking data dimensions from the first optics group: 2
 + Taking angpix from the first optics group: 1.059
 + Back-projecting all images ...
  13/  13 sec ............................................................~~(,_,">
 + Starting the reconstruction ...
 + Done! Written output map in: CHX_data/r

We will now invert and low-pass filter

In [7]:
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-only_half1.mrc --o CHX_data/reconstructions/reconstruct_60S-only_half1_invert.mrc --multiply_constant -1
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-only_half2.mrc --o CHX_data/reconstructions/reconstruct_60S-only_half2_invert.mrc --multiply_constant -1

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-rotated_half1.mrc --o CHX_data/reconstructions/reconstruct_60S-rotated_half1_invert.mrc --multiply_constant -1
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-rotated_half2.mrc --o CHX_data/reconstructions/reconstruct_60S-rotated_half2_invert.mrc --multiply_constant -1

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_half1.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_half1_invert.mrc --multiply_constant -1
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_half2.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_half2_invert.mrc --multiply_constant -1

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-all_half1.mrc --o CHX_data/reconstructions/reconstruct_60S-all_half1_invert.mrc --multiply_constant -1
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-all_half2.mrc --o CHX_data/reconstructions/reconstruct_60S-all_half2_invert.mrc --multiply_constant -1

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-only_all.mrc --o CHX_data/reconstructions/reconstruct_60S-only_ctf_invert_lp30.mrc --lowpass 30 --multiply_constant -1

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-rotated_all.mrc --o CHX_data/reconstructions/reconstruct_60S-rotated_ctf_invert_lp30.mrc --lowpass 30 --multiply_constant -1

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_all.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_ctf_invert_lp30.mrc --lowpass 30 --multiply_constant -1

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-all_all.mrc --o CHX_data/reconstructions/reconstruct_60S-all_ctf_invert_lp30.mrc --lowpass 30 --multiply_constant -1


   4/   4 sec ............................................................~~(,_,">
   4/   4 sec ............................................................~~(,_,">
   4/   4 sec ............................................................~~(,_,">
   4/   4 sec ............................................................~~(,_,">
   4/   4 sec ............................................................~~(,_,">
   5/   5 sec ............................................................~~(,_,">
   4/   4 sec ............................................................~~(,_,">
   4/   4 sec ............................................................~~(,_,">
000/??? sec ~~(,_,">                                                          [oo]WARNING: You did not specify --angpix. The pixel size in the image header, 1.059 A/px, is used.
   8/   8 sec ............................................................~~(,_,">
000/??? sec ~~(,_,">                                                       

Let's do the masking again

In [9]:
!relion_mask_create --denovo --box_size 512 --outer_radius 400 --o CHX_data/masks/spherical_mask_wide.mrc --j 16

 Done creating mask! Written out: CHX_data/masks/spherical_mask_wide.mrc


In [10]:
!relion_mask_create --i CHX_data/reconstructions/reconstruct_60S-only_ctf_invert_lp30.mrc --o CHX_data/masks/60S_only_mask.mrc --extend_inimask 6 --width_soft_edge 6 --ini_threshold 0.002 --j 20

!relion_mask_create --i CHX_data/reconstructions/reconstruct_60S-rotated_ctf_invert_lp30.mrc --o CHX_data/masks/60S_rotated_mask.mrc --extend_inimask 6 --width_soft_edge 6 --ini_threshold 0.0025 --j 20

!relion_mask_create --i CHX_data/reconstructions/reconstruct_60S-nonrotated_ctf_invert_lp30.mrc --o CHX_data/masks/60S_nonrotated_mask.mrc --extend_inimask 6 --width_soft_edge 6 --ini_threshold 0.0026 --j 20

!relion_mask_create --i CHX_data/reconstructions/reconstruct_60S-all_ctf_invert_lp30.mrc --o CHX_data/masks/60S_all_mask.mrc --extend_inimask 6 --width_soft_edge 6 --ini_threshold 0.002 --j 20




 Creating a mask ...
         The value in the input image header (= 1.059) is used instead.
== Extending initial binary mask ...
  24/  24 sec ............................................................~~(,_,">
== Making a soft edge on the extended mask ...
  22/  22 sec ............................................................~~(,_,">
 Done creating mask! Written out: CHX_data/masks/60S_only_mask.mrc
 Creating a mask ...
         The value in the input image header (= 1.059) is used instead.
== Extending initial binary mask ...
  25/  25 sec ............................................................~~(,_,">
== Making a soft edge on the extended mask ...
  23/  23 sec ............................................................~~(,_,">
 Done creating mask! Written out: CHX_data/masks/60S_rotated_mask.mrc
 Creating a mask ...
         The value in the input image header (= 1.059) is used instead.
== Extending initial binary mask ...
  24/  24 sec .................................

In [14]:
!relion_mask_create --i CHX_data/reconstructions/reconstruct_60S-nonrotated_ctf_invert_lp30.mrc --o CHX_data/masks/60S_nonrotated_mask.mrc --extend_inimask 8 --width_soft_edge 8 --ini_threshold 0.0031 --j 20

 Creating a mask ...
         The value in the input image header (= 1.059) is used instead.
== Extending initial binary mask ...
  55/  55 sec ............................................................~~(,_,">
== Making a soft edge on the extended mask ...
  55/  55 sec ............................................................~~(,_,">
 Done creating mask! Written out: CHX_data/masks/60S_nonrotated_mask.mrc


In [15]:
import mrcfile
import numpy as np
from pathlib import Path
import sys

sys.argv = [
    'multiply_masks_with_spherical.py',
    'CHX_data/masks/',
    'CHX_data/masks/spherical_mask_wide.mrc'
]

def create_spherical_mask(size, radius=None):
    """Create a spherical mask with soft edges."""
    if radius is None:
        radius = size // 2 - 2  # Default to slightly smaller than half the box
    
    # Create coordinate arrays
    x = np.arange(size) - size // 2
    y = np.arange(size) - size // 2
    z = np.arange(size) - size // 2
    X, Y, Z = np.meshgrid(x, y, z)
    
    # Calculate distance from center
    R = np.sqrt(X**2 + Y**2 + Z**2)
    
    # Create soft-edged mask
    mask = np.ones_like(R, dtype=np.float32)
    edge_width = 5  # Width of soft edge in pixels
    mask[R > radius] = 0
    edge_mask = (R > (radius - edge_width)) & (R <= radius)
    mask[edge_mask] = 0.5 * (1 + np.cos(np.pi * (R[edge_mask] - (radius - edge_width)) / edge_width))
    
    return mask

def multiply_masks_with_spherical(input_dir, spherical_mask_path, output_dir=None):
    """
    Multiply category masks with an existing spherical mask.
    
    Args:
        input_dir (str): Directory containing the input masks
        spherical_mask_path (str): Path to the spherical mask file
        output_dir (str, optional): Directory to save output masks. If None, uses input_dir
    """
    if output_dir is None:
        output_dir = input_dir
    
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Specific mask files to process
    mask_files = [
        '60S_all_mask.mrc',
        '60S_nonrotated_mask.mrc',
        '60S_rotated_mask.mrc',
        '60S_only_mask.mrc'
    ]
    
    # Load the spherical mask
    print(f"Loading spherical mask from {spherical_mask_path}")
    with mrcfile.open(spherical_mask_path) as mrc:
        spherical_mask = mrc.data
    
    # Process each mask
    for mask_name in mask_files:
        mask_path = Path(input_dir) / mask_name
        if not mask_path.exists():
            print(f"Warning: Mask not found: {mask_path}")
            continue
        
        print(f"Processing {mask_path}")
        
        # Read the mask
        with mrcfile.open(mask_path) as mrc:
            mask_data = mrc.data
        
        # Multiply with spherical mask
        multiplied_mask = mask_data * spherical_mask
        
        # Save the result
        output_path = Path(output_dir) / f"{mask_path.stem}_mult.mrc"
        with mrcfile.new(output_path, overwrite=True) as mrc:
            mrc.set_data(multiplied_mask)
        
        print(f"Saved multiplied mask to {output_path}")

if __name__ == "__main__":
    import argparse
    
    parser = argparse.ArgumentParser(description="Multiply category masks with a spherical mask")
    parser.add_argument("input_dir", help="Directory containing the input masks")
    parser.add_argument("spherical_mask", help="Path to the spherical mask file")
    parser.add_argument("--output-dir", help="Directory to save output masks (default: same as input)")
    
    args = parser.parse_args()
    multiply_masks_with_spherical(args.input_dir, args.spherical_mask, args.output_dir) 

Loading spherical mask from CHX_data/masks/spherical_mask_wide.mrc
Processing CHX_data/masks/60S_all_mask.mrc
Saved multiplied mask to CHX_data/masks/60S_all_mask_mult.mrc
Processing CHX_data/masks/60S_nonrotated_mask.mrc
Saved multiplied mask to CHX_data/masks/60S_nonrotated_mask_mult.mrc
Processing CHX_data/masks/60S_rotated_mask.mrc
Saved multiplied mask to CHX_data/masks/60S_rotated_mask_mult.mrc
Processing CHX_data/masks/60S_only_mask.mrc
Saved multiplied mask to CHX_data/masks/60S_only_mask_mult.mrc


Postprocess

In [12]:
!relion_postprocess --i CHX_data/reconstructions/reconstruct_60S-rotated_half1_invert.mrc --o CHX_data/reconstructions/reconstruct_60S-rotated_postprocessed --mask CHX_data/masks/60S_rotated_mask_mult.mrc --mtf mtf_falcon4EC_300kV.star --auto_bfac

!relion_postprocess --i CHX_data/reconstructions/reconstruct_60S-nonrotated_half1_invert.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed --mask CHX_data/masks/60S_nonrotated_mask_mult.mrc --mtf mtf_falcon4EC_300kV.star --auto_bfac

!relion_postprocess --i CHX_data/reconstructions/reconstruct_60S-only_half1_invert.mrc --o CHX_data/reconstructions/reconstruct_60S-only_postprocessed --mask CHX_data/masks/60S_only_mask_mult.mrc --mtf mtf_falcon4EC_300kV.star --auto_bfac

!relion_postprocess --i CHX_data/reconstructions/reconstruct_60S-all_half1_invert.mrc --o CHX_data/reconstructions/reconstruct_60S-all_postprocessed --mask CHX_data/masks/60S_all_mask_mult.mrc --mtf mtf_falcon4EC_300kV.star --auto_bfac




== Reading input half-reconstructions: 
  + half1-map:                     CHX_data/reconstructions/reconstruct_60S-rotated_half1_invert.mrc
  + half2-map:                     CHX_data/reconstructions/reconstruct_60S-rotated_half2_invert.mrc
 + --mtf_angpix was not provided, assuming pixel size in raw micrographs is the same as in particles, 1.059 A/px.
== Using a user-provided mask ... 
  + input mask:                    CHX_data/masks/60S_rotated_mask_mult.mrc
  + fraction f (solvent mask based): 12.5024
  + molecular weight inside protein mask: 4.91582e+06
== Masking input maps ...
  + randomize phases beyond:       108.442 Angstroms
== Dividing map by the MTF of the detector ...
  + mtf STAR-file:                 mtf_falcon4EC_300kV.star
== Applying sqrt(2*FSC/(FSC+1)) weighting (as in Rosenthal & Henderson, 2003) ...
== Fitting straight line through Guinier plot to find B-factor ...
  + fit from resolution:           10
  + fit until resolution:          0
  + slope of fit:       

In [16]:
!relion_postprocess --i CHX_data/reconstructions/reconstruct_60S-nonrotated_half1_invert.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed --mask CHX_data/masks/60S_nonrotated_mask_mult.mrc --mtf mtf_falcon4EC_300kV.star --auto_bfac

== Reading input half-reconstructions: 
  + half1-map:                     CHX_data/reconstructions/reconstruct_60S-nonrotated_half1_invert.mrc
  + half2-map:                     CHX_data/reconstructions/reconstruct_60S-nonrotated_half2_invert.mrc
 + --mtf_angpix was not provided, assuming pixel size in raw micrographs is the same as in particles, 1.059 A/px.
== Using a user-provided mask ... 
  + input mask:                    CHX_data/masks/60S_nonrotated_mask_mult.mrc
  + fraction f (solvent mask based): 11.9773
  + molecular weight inside protein mask: 5.13132e+06
== Masking input maps ...
  + randomize phases beyond:       180.736 Angstroms
== Dividing map by the MTF of the detector ...
  + mtf STAR-file:                 mtf_falcon4EC_300kV.star
== Applying sqrt(2*FSC/(FSC+1)) weighting (as in Rosenthal & Henderson, 2003) ...
== Fitting straight line through Guinier plot to find B-factor ...
  + fit from resolution:           10
  + fit until resolution:          0
  + slope of fi

lp filter

In [13]:
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_lp8.mrc --lowpass 8
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_lp20.mrc --lowpass 20

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-rotated_postprocessed.mrc --o CHX_data/reconstructions/reconstruct_60S-rotated_postprocessed_lp8.mrc --lowpass 8
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-rotated_postprocessed.mrc --o CHX_data/reconstructions/reconstruct_60S-rotated_postprocessed_lp20.mrc --lowpass 20

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-only_postprocessed.mrc --o CHX_data/reconstructions/reconstruct_60S-only_postprocessed_lp8.mrc --lowpass 8
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-only_postprocessed.mrc --o CHX_data/reconstructions/reconstruct_60S-only_postprocessed_lp20.mrc --lowpass 20

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-all_postprocessed.mrc --o CHX_data/reconstructions/reconstruct_60S-all_postprocessed_lp8.mrc --lowpass 8
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-all_postprocessed.mrc --o CHX_data/reconstructions/reconstruct_60S-all_postprocessed_lp20.mrc --lowpass 20

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_masked.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_masked_lp8.mrc --lowpass 8
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_masked.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_masked_lp20.mrc --lowpass 20

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-rotated_postprocessed_masked.mrc --o CHX_data/reconstructions/reconstruct_60S-rotated_postprocessed_masked_lp8.mrc --lowpass 8
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-rotated_postprocessed_masked.mrc --o CHX_data/reconstructions/reconstruct_60S-rotated_postprocessed_masked_lp20.mrc --lowpass 20

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-only_postprocessed_masked.mrc --o CHX_data/reconstructions/reconstruct_60S-only_postprocessed_masked_lp8.mrc --lowpass 8
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-only_postprocessed_masked.mrc --o CHX_data/reconstructions/reconstruct_60S-only_postprocessed_masked_lp20.mrc --lowpass 20

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-all_postprocessed_masked.mrc --o CHX_data/reconstructions/reconstruct_60S-all_postprocessed_masked_lp8.mrc --lowpass 8
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-all_postprocessed_masked.mrc --o CHX_data/reconstructions/reconstruct_60S-all_postprocessed_masked_lp20.mrc --lowpass 20


000/??? sec ~~(,_,">                                                          [oo]WARNING: You did not specify --angpix. The pixel size in the image header, 1.059 A/px, is used.
   8/   8 sec ............................................................~~(,_,">
000/??? sec ~~(,_,">                                                          [oo]WARNING: You did not specify --angpix. The pixel size in the image header, 1.059 A/px, is used.
   8/   8 sec ............................................................~~(,_,">
000/??? sec ~~(,_,">                                                          [oo]WARNING: You did not specify --angpix. The pixel size in the image header, 1.059 A/px, is used.
   9/   9 sec ............................................................~~(,_,">
000/??? sec ~~(,_,">                                                          [oo]WARNING: You did not specify --angpix. The pixel size in the image header, 1.059 A/px, is used.
   9/   9 sec .........................

In [17]:
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_lp8.mrc --lowpass 8
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_lp20.mrc --lowpass 20

!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_masked.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_masked_lp8.mrc --lowpass 8
!relion_image_handler --i CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_masked.mrc --o CHX_data/reconstructions/reconstruct_60S-nonrotated_postprocessed_masked_lp20.mrc --lowpass 20

000/??? sec ~~(,_,">                                                          [oo]WARNING: You did not specify --angpix. The pixel size in the image header, 1.059 A/px, is used.
  10/  10 sec ............................................................~~(,_,">
000/??? sec ~~(,_,">                                                          [oo]WARNING: You did not specify --angpix. The pixel size in the image header, 1.059 A/px, is used.
  11/  11 sec ............................................................~~(,_,">
000/??? sec ~~(,_,">                                                          [oo]WARNING: You did not specify --angpix. The pixel size in the image header, 1.059 A/px, is used.
   9/   9 sec ............................................................~~(,_,">
000/??? sec ~~(,_,">                                                          [oo]WARNING: You did not specify --angpix. The pixel size in the image header, 1.059 A/px, is used.
  10/  10 sec .........................